# Test bot

In [1]:
from ollama_models import ollama_models
models = ollama_models()    

In [2]:
hard_ids = [6614, 1482, 1337, 7811, 8734, 5531, 1454, 6633, 2622, 3054]

In [3]:
from load_saved_questions import load_saved_questions
questions = load_saved_questions(hard_ids)

2025-06-18 14:32:08.048 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [4]:
from community_forecast import community_forecast
id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}

In [ ]:
import pandas as pd
from flatten_dict import flatten_dict
from datetime import datetime
df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
df['id_of_post'] = [q.id_of_post for q in questions]
df['id_of_question'] = [q.id_of_question for q in questions]
df['question_options'] = df['question_options'].apply(repr)
df['today'] = datetime.now().strftime("%Y-%m-%d")
df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)
df = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title', 'question_description',
    'question_resolution_criteria', 'question_fine_print', 'question_type',
    'question_options', 'question_group_variable', 'question_question_weight',
    'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
    'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']]

In [7]:
from gather_research_and_set_prompt import gather_research_and_set_prompt
live=True
df, rag = gather_research_and_set_prompt(df, live)

Loaded existing index from forecast_index.faiss
Index contains 4767 vectors at initialization


In [ ]:
import os
from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
for model in models:
    print("Model", model)
    dn = f'forecasts_{model}'
    os.makedirs(dn, exist_ok=True)
    df = generate_forecasts_and_update_rag(df, rag, live, model)
    print(df)